# HW#4&5 MRET HEIN SQL Propositions


### Proposition 1: Return all orders sold by salespersons with the highest number of orders



**Business Value:** 
- Helps recognize top-performing salespeople based on activity volume.
- Lets managers analyze what kinds of orders or customers those high performers handle.
- Can inform training and performance reviews by comparing top reps’ patterns to others.

-- Step 1: Find the salesperson with the highest number of orders
    --The first CTE (SalesCounts) counts how many orders each salesperson handled.
    --The second CTE (TopSalespersons) finds the salesperson(s) with the maximum order count.
-- Step 2: Return all orders handled by that salesperson/those salespersons

In [ ]:
WITH SalesCounts AS ( 
    SELECT
        SalespersonPersonID,
        COUNT(OrderID) AS OrderCount
    FROM Sales.Orders
    GROUP BY SalespersonPersonID
),
TopSalespersons AS (
    SELECT SalespersonPersonID
    FROM SalesCounts
    WHERE OrderCount = (
        SELECT MAX(OrderCount) FROM SalesCounts
    )
)
SELECT 
    o.OrderID,
    o.OrderDate,
    o.CustomerID,
    c.CustomerName,
    p.FullName AS Salesperson
FROM Sales.Orders AS o
JOIN Sales.Customers AS c ON c.CustomerID = o.CustomerID
JOIN Application.People AS p ON p.PersonID = o.SalespersonPersonID
WHERE o.SalespersonPersonID IN (SELECT SalespersonPersonID FROM TopSalespersons)
ORDER BY o.OrderDate DESC, o.OrderID;

### Proposition 2: All the sales persons who do not have orders on or after Jan 1st, 2022

In [ ]:
**Business Value:** 
-Sales Management: Identify inactive or underperforming sales reps for coaching or re-assignment.
-HR/Finance: Review compensation plans and evaluate team performance.
-Operations: Plan territory coverage—regions without active salespeople may need reallocation.

In [ ]:
SELECT 
    p.PersonID,
    p.FullName AS Salesperson
FROM Application.People AS p
WHERE p.IsSalesperson = 1
  AND NOT EXISTS (
        SELECT 1
        FROM Sales.Orders AS o
        WHERE o.SalespersonPersonID = p.PersonID
          AND o.OrderDate >= '2022-01-01'
     )
ORDER BY p.FullName;


### Proposition 3: Countries that are delivered by Air Freight but not Road Frieght

**Business Value:** 
-Helps logistics and operations identify countries where delivery van is the only active method as compared to air freight whichh is useful for cost analysis or introducing new delivery modes.
-Finance can assess shipping cost exposure (air freight is expensive).
-Customer service can set accurate delivery expectations for those regions.

In [ ]:
SELECT DISTINCT co.CountryName, co.Region , co.LatestRecordedPopulation
FROM Application.Countries AS co
WHERE EXISTS (
    SELECT 1
    FROM Sales.Customers AS c
    JOIN Application.Cities AS ci ON ci.CityID = c.DeliveryCityID
    JOIN Application.StateProvinces AS sp ON sp.StateProvinceID = ci.StateProvinceID
    WHERE sp.CountryID = co.CountryID
      AND c.DeliveryMethodID = (
          SELECT DeliveryMethodID 
          FROM Application.DeliveryMethods
          WHERE DeliveryMethodName = 'Delivery Van'
      )
)
AND NOT EXISTS (
    SELECT 1
    FROM Sales.Customers AS c
    JOIN Application.Cities AS ci ON ci.CityID = c.DeliveryCityID
    JOIN Application.StateProvinces AS sp ON sp.StateProvinceID = ci.StateProvinceID
    WHERE sp.CountryID = co.CountryID
      AND c.DeliveryMethodID = (
          SELECT DeliveryMethodID 
          FROM Application.DeliveryMethods
          WHERE DeliveryMethodName = 'Air Freight'
      )
)
ORDER BY co.CountryName;

### Proposition 4: All orders placed on the sales persons's last day of activity for each sales person

**Business Value:** 
-- Helps HR and management with guiding new employees to resume activity of the sales person
-- Investigate the sales person perfomance before he/she is let go or quit
-- Retain the customer base of that sales person

In [ ]:
WITH LastActiveDay AS (
  SELECT
      o.SalespersonPersonID,
      CAST(MAX(o.OrderDate) AS date) AS LastOrderDay
  FROM Sales.Orders AS o
  GROUP BY o.SalespersonPersonID
)
SELECT
  p.FullName        AS Salesperson,
  o.OrderID,
  o.OrderDate,
  c.CustomerID,
  c.CustomerName
FROM LastActiveDay AS lad
JOIN Sales.Orders   AS o  ON o.SalespersonPersonID = lad.SalespersonPersonID
                         AND o.OrderDate >= lad.LastOrderDay
                         AND o.OrderDate <  DATEADD(day, 1, lad.LastOrderDay)  -- same calendar day
JOIN Application.People  AS p  ON p.PersonID   = lad.SalespersonPersonID
JOIN Sales.Customers     AS c  ON c.CustomerID = o.CustomerID
WHERE p.IsSalesperson = 1
ORDER BY p.FullName, o.OrderDate, o.OrderID;

### Proposition 5: Returns stock items that are sold in 2016 but not 2017

**Business Value:** 
-Inventory management – to identify products that have stopped selling recently.
-Marketing & Product teams – to investigate whether those items were discontinued or replaced.
-Finance – to track revenue gaps or forecast obsolescence.

In [ ]:
SELECT DISTINCT si.StockItemID,
       si.StockItemName
FROM Warehouse.StockItems AS si
WHERE EXISTS (
    SELECT 1
    FROM Sales.Invoices AS i
    JOIN Sales.InvoiceLines AS il ON il.InvoiceID = i.InvoiceID
    WHERE il.StockItemID = si.StockItemID
      AND YEAR(i.InvoiceDate) = 2016
)
AND NOT EXISTS (
    SELECT 1
    FROM Sales.Invoices AS i
    JOIN Sales.InvoiceLines AS il ON il.InvoiceID = i.InvoiceID
    WHERE il.StockItemID = si.StockItemID
      AND YEAR(i.InvoiceDate) = 2017
)
ORDER BY si.StockItemName;

### Proposition 6: Filter orders that were not delivered on the last day of the year

**Business Value:** 
- Use for logistics to circumvent unforseen circumstances
- Can be used to track orders that are unassociated and not delivered during certain events

In [ ]:
-- Orders delivered, with delivery date normalized to DATE
WITH Delivered AS (
    SELECT
        o.OrderID,
        o.OrderDate,
        CAST(i.ConfirmedDeliveryTime AS date) AS DeliveryDate
    FROM Sales.Orders AS o
    JOIN Sales.Invoices AS i
      ON i.OrderID = o.OrderID
    WHERE i.ConfirmedDeliveryTime IS NOT NULL
),
DeliveryYear AS (
    SELECT
        d.*,
        DATEFROMPARTS(YEAR(d.DeliveryDate), 12, 31) AS YearEndDate
    FROM Delivered AS d
)
SELECT
    OrderID,
    OrderDate,
    DeliveryDate
FROM DeliveryYear
WHERE DeliveryDate <> YearEndDate        --  keep orders NOT delivered on Dec 31 of that year
ORDER BY DeliveryDate DESC, OrderID;

### Proposition 7: Create a view that returns the total quantity of orders for each salesperson and year

**Business Value:** 
-Sales Management: Track how each salesperson’s total sales quantity trends by year.
-Finance & Planning: Forecast production and staffing needs based on yearly sales volume.
-HR & Incentives: Support annual performance reviews and commission calculations.

In [ ]:
CREATE OR ALTER VIEW Sales.vTotalQuantityBySalespersonYear
AS
SELECT
    p.FullName                AS Salesperson,
    YEAR(o.OrderDate)         AS OrderYear,
    SUM(ol.Quantity)          AS TotalQuantity
FROM Sales.Orders AS o
JOIN Sales.OrderLines  AS ol ON ol.OrderID = o.OrderID
JOIN Application.People AS p  ON p.PersonID = o.SalespersonPersonID
WHERE p.IsSalesperson = 1
GROUP BY p.FullName, YEAR(o.OrderDate);


In [ ]:
--USE THE VIEW
SELECT *
FROM Sales.vTotalQuantityBySalespersonYear
ORDER BY OrderYear DESC, TotalQuantity DESC;

### Proposition 8: Return the running quantity for each stock item and year using subqueries

**Business Value:** 
-Tracks daily or monthly stock movement for each item.
-Helps predict when inventory will deplete, so purchasing can reorder before stockouts happen
-Offers a trend view that supports long-term decisions — when to ramp up production, retire products, or shift storage capacity.

In [ ]:
-- Running quantity per Stock Item and Year (subquery-only approach)
WITH ItemDaily AS (
  SELECT
      si.StockItemID,
      si.StockItemName,
      CAST(i.InvoiceDate AS date) AS SalesDate,
      YEAR(i.InvoiceDate)         AS OrderYear,
      SUM(CASE WHEN i.IsCreditNote = 1
               THEN -il.Quantity
               ELSE  il.Quantity END) AS DailyQty
  FROM Sales.InvoiceLines AS il
  JOIN Sales.Invoices     AS i  ON i.InvoiceID     = il.InvoiceID
  JOIN Warehouse.StockItems AS si ON si.StockItemID = il.StockItemID
  GROUP BY
      si.StockItemID,
      si.StockItemName,
      CAST(i.InvoiceDate AS date),
      YEAR(i.InvoiceDate)
)
SELECT
    d.StockItemID,
    d.StockItemName,
    d.OrderYear,
    d.SalesDate,
    d.DailyQty,
    -- Correlated subquery does the running (cumulative) total within item+year
    (
      SELECT SUM(d2.DailyQty)
      FROM ItemDaily AS d2
      WHERE d2.StockItemID = d.StockItemID
        AND d2.OrderYear   = d.OrderYear
        AND d2.SalesDate  <= d.SalesDate
    ) AS RunningQty
FROM ItemDaily AS d
ORDER BY
    d.StockItemName,
    d.OrderYear,
    d.SalesDate;


### Proposition 9: Return for each order the number of days that past since the same salesperson's previous order

**Business Value:** 
-Find reps who go long stretches without any sales 
—Track inactive accounts or lost focus.
-Identify top performers who maintain steady order frequency.

In [ ]:
SELECT
  p.FullName AS Salesperson,
  o.OrderID,
  o.OrderDate,
  (SELECT MAX(o2.OrderDate)
   FROM Sales.Orders AS o2
   WHERE o2.SalespersonPersonID = o.SalespersonPersonID
     AND o2.OrderDate < o.OrderDate) AS PrevOrderDate,
  DATEDIFF(day,
    (SELECT MAX(o2.OrderDate)
     FROM Sales.Orders AS o2
     WHERE o2.SalespersonPersonID = o.SalespersonPersonID
       AND o2.OrderDate < o.OrderDate),
    o.OrderDate) AS DaysSincePrevOrder
FROM Sales.Orders AS o
JOIN Application.People AS p ON p.PersonID = o.SalespersonPersonID
ORDER BY Salesperson, o.OrderDate, o.OrderID;


### Proposition 10: View that computes net revenue by city (deliveries minus returns)

**Business Value:** 
-By ranking cities by net revenue, teams can see where customers generate the most income. This helps target campaigns, allocate budgets, and plan local promotions where returns justify effort.
-Reveals geographic demand clusters so warehousing and shipping routes can be optimized. Cities with high net revenue may deserve priority delivery lanes or stocking points.

In [ ]:
--CREATE VIEW
CREATE OR ALTER VIEW Sales.vNetRevenueByCity
AS
SELECT
    ci.CityName,
    co.CountryName,                             
    SUM(CASE WHEN i.IsCreditNote = 1
             THEN -il.ExtendedPrice
             ELSE  il.ExtendedPrice
        END) AS NetRevenue
FROM Sales.InvoiceLines AS il
JOIN Sales.Invoices      AS i   ON i.InvoiceID   = il.InvoiceID
JOIN Sales.Customers     AS c   ON c.CustomerID  = i.CustomerID
JOIN Application.Cities  AS ci  ON ci.CityID     = COALESCE(c.DeliveryCityID, c.PostalCityID)
JOIN Application.StateProvinces AS sp ON sp.StateProvinceID = ci.StateProvinceID
JOIN Application.Countries      AS co ON co.CountryID       = sp.CountryID
GROUP BY ci.CityName, co.CountryName;


In [ ]:
--USE THE VIEW
SELECT CityName, CountryName, NetRevenue
FROM Sales.vNetRevenueByCity
ORDER BY NetRevenue DESC, CityName;
